# Indexy

- **Index** je neměnná (immutable) sekvence, která slouží k označení, indexaci a řazení dat v pandas objektech, jako jsou **Series** a **DataFrame**.
- **Index** může obsahovat duplicitní hodnoty, což znamená, že více řádků může mít stejný index.
- Může obsahovat různé datové typy (např. čísla, texty, datumy) a pracuje s nimi efektivně.
- **MultiIndex** (víceúrovňový index) je struktura indexu, která umožňuje použití více úrovní indexu. Umožňuje lepší organizaci a hierarchii dat, zejména při práci se složitějšími datovými sadami.

## Vybrané metody a atributy pro práci s indexem

### Atributy
- **`Index.values`**: Vrací hodnoty indexu jako numpy pole. Viz [Index.values](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.values.html).
- **`Index.nlevels`**: Informace o počtu úrovní indexu. Používá se především u **MultiIndex**. Viz [Index.nlevels](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.nlevels.html).
- **`Index.is_unique`**: Ověří, zda jsou hodnoty indexu unikátní. Pokud nejsou, metoda vrací `False`. Viz [Index.is_unique](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.is_unique.html).
- **`Index.hasnans`**: Zjistí, zda se v indexu nachází hodnota `NaN`. Pokud ano, vrací `True`. Viz [Index.hasnans](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.hasnans.html).

### Metody
- **`reset_index()`**: Resetuje aktuální index **DataFrame** a převede jej zpět na výchozí číselný index. Původní index může být volitelně přidán jako nový sloupec. Viz [reset_index()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html).
- **`set_index()`**: Použije existující sloupec (nebo více sloupců) jako nový index. Je to užitečné pro změnu struktury dat. Viz [set_index()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html).
- **`sort_index()`**: Seřadí hodnoty indexu podle abecedy nebo číselně. Může být použit jak na řádkový, tak na sloupcový index. Viz [sort_index()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_index.html).


In [ ]:
# Import potrebnych knihoven
import pandas as pd

# Stazeni .csv souboru publikovaneho na github.com
url = 'https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv'
df = pd.read_csv(url,delimiter = ';')
df['DATE'] = pd.to_datetime(df['DATE'])
df.index.is_unique

True

## Multiindex

- **MultiIndex** je víceúrovňový index, který umožňuje přiřadit více než jednu úroveň indexu ke každému řádku nebo sloupci. Umožňuje reprezentovat data s hierarchickou strukturou, což je užitečné při práci se složitými daty, například při shlukování nebo agregaci dat.
- Víceúrovňové indexy se často používají při seskupování dat (`groupby()`) přes více sloupců nebo při vytváření kontingenčních tabulek s metodou `pivot_table()`.

## Vybrané metody pro práci s multiindexem

- **`droplevel()`**: Odstraní jednu nebo více úrovní **MultiIndex**. Je užitečné, pokud potřebujete zjednodušit hierarchii indexu. Viz [droplevel()](https://pandas.pydata.org/docs/reference/api/pandas.MultiIndex.droplevel.html).
- **`sortlevel()`**: Seřadí **MultiIndex** podle vybrané úrovně. Toto třídění lze provést vzestupně nebo sestupně. Viz [sortlevel()](https://pandas.pydata.org/docs/reference/api/pandas.MultiIndex.sortlevel.html).
- **`reorder_levels()`**: Umožňuje změnit pořadí úrovní v **MultiIndex**. Viz [reorder_levels()](https://pandas.pydata.org/docs/reference/api/pandas.MultiIndex.reorder_levels.html).


# Práce s MultiIndexem
grouped.index.nlevels  # Počet úrovní indexu
grouped.reset_index()   # Resetování MultiIndexu
grouped.sort_index(level=0)  # Seřazení podle první úrovně indexu


In [ ]:
srazky = df.copy()
# Nastaveni multiindexu
srazky = srazky.set_index(['DATE','NAME','STATION'])
# Zmena poradi urovni multiindexu
srazky = srazky.reorder_levels(['NAME','STATION','DATE'])
# Odstraneni multiindexu na urovni 1
srazky = srazky.droplevel(1,axis=0)
srazky

PRCP  SNWD  TAVG  TMAX  TMIN
NAME   DATE                                    
PIDING 2006-12-01   0.0   0.0   NaN   4.2   1.0
       2006-12-02   0.0   0.0   NaN   1.8  -1.7
       2006-12-03   0.0   0.0   NaN   5.1  -4.6
       2006-12-04  19.0   0.0   NaN   8.3  -0.9
       2006-12-05   0.0   0.0   NaN  12.0   0.9
...                 ...   ...   ...   ...   ...
TURANY 2020-11-14   0.0   NaN   6.2   NaN   NaN
       2020-11-15   0.0   NaN   6.6   NaN   5.7
       2020-11-16   0.5   NaN   5.9   NaN   NaN
       2020-11-17   0.0   NaN   6.5   NaN   1.7
       2020-11-18   0.0   NaN   7.9  12.7   NaN

[67834 rows x 5 columns]

In [ ]:
srazky1  = df.copy()
srazky1['ROK'] = srazky1['DATE'].dt.year
srazky1['MESIC'] = srazky1['DATE'].dt.month
# Agregace dat za pomoci kontingencni tabulky s nastavenim sloupcu jako indexu
srazky1 = srazky1.pivot_table(values= ['PRCP'],index=['ROK','MESIC'],columns='NAME',aggfunc='sum',fill_value='-')
srazky1.droplevel(0,axis=0)


C:\Users\Student\AppData\Local\Temp\ipykernel_13940\1216921114.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  srazky1 = srazky1.pivot_table(values= ['PRCP'],index=['ROK','MESIC'],columns='NAME',aggfunc='sum',fill_value='-')


PRCP                                                            \
NAME    GRAZ KOSICE MOSNOV PIDING PRAHA KLEMENTINUM PRAHA LIBUS RUZYNE   
MESIC                                                                    
1        9.2   18.9   30.7      -              17.0        29.4   24.7   
2        0.5   36.6   20.8      -              18.3        18.5   23.9   
3       67.6   31.6   31.2      -              81.4        48.5   42.3   
4       18.1   43.4   40.4      -               6.5         2.9    7.4   
5       90.9   57.3   71.7      -              38.7        10.9   29.1   
...      ...    ...    ...    ...               ...         ...    ...   
7       82.1   91.0   82.2  175.4                 -        21.5   34.5   
8      227.0   62.8   86.9  257.9                 -       129.4  138.0   
9       79.0   63.5  150.4  112.6                 -        65.3   58.9   
10      38.9  107.6  162.9  155.2                 -        58.4   70.3   
11      13.4    7.9   19.7      -                 -        10.5    7.3   

                              
NAME  SALZBURG TURANY   WIEN  
MESIC                         
1         51.2   34.0   43.7  
2         66.1   22.6   35.1  
3        154.2   15.5   80.4  
4         30.5    5.1   11.1  
5        115.3   33.4   50.4  
...        ...    ...    ...  
7          0.0   55.3   27.1  
8          0.0  104.5   33.5  
9          0.0   75.8   91.2  
10         8.6   95.0  120.9  
11         0.0   20.4    7.1  

[251 rows x 10 columns]

### Cvičení 1 zadání
1. Z dat dostupných pod touto URL vytvořte DataFrame s názvem `df`: ['https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv'](https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv)
2. Filtrujte `df` na `NAME == 'RUZYNE'`.
3. Dosaďte sloupec `DATE` do indexu.
4. Zjistěte, jestli index má `NaN` hodnoty a jestli obsahuje duplicity.


In [25]:
import pandas as pd 

df = pd.read_csv(r'https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv',sep=';')
df = df.loc[df['NAME'] == 'RUZYNE']
df = df.set_index(df['DATE'])
print(df.index.hasnans)
print(df.index.is_unique)

False
True


### Cvičení 2 zadání
1. Z dat dostupných pod touto URL vytvořte DataFrame s názvem `df`: ['https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv'](https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv)
2. Změňte datový typ sloupce `DATE` na `datetime`.
3. Do nového sloupce `ROK` spočítejte rok ze sloupce `DATE`.
4. Do nového sloupce `MESIC` spočítejte měsíc ze sloupce `DATE`.
5. Agregujte DataFrame dle sloupců `NAME` a `ROK`, hodnoty budou suma `PRCP`.
6. Upravte multiindex tak, aby měl toto pořadí: `ROK`, `NAME`.
7. Seřaďte index `ROK` vzestupně.


In [20]:
import pandas as pd 

df = pd.read_csv(r'https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv',sep=';')
df['DATE'] = pd.to_datetime(df['DATE'])
df['ROK'] = df['DATE'].dt.year
df['MESIC'] = df['DATE'].dt.month
df = df.groupby(by=['ROK','NAME'])['PRCP'].sum()
df = df.reorder_levels(['ROK','NAME'])
df.sort_index(level=0,ascending=True)
df

ROK   NAME             
2000  GRAZ                 755.3
      KOSICE               530.8
      MOSNOV               502.3
      PRAHA KLEMENTINUM    385.9
      PRAHA LIBUS          163.6
                           ...  
2020  PRAHA LIBUS          611.1
      RUZYNE               547.1
      SALZBURG               8.6
      TURANY               610.6
      WIEN                 499.3
Name: PRCP, Length: 189, dtype: float64

In [1]:
import ipywidgets as widgets
# Import potrebnych knihoven
import pandas as pd

# Stazeni .csv souboru publikovaneho na github.com
url = 'https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv'
df = pd.read_csv(url,delimiter = ';')
df['DATE'] = pd.to_datetime(df['DATE'])

# Dopocitan sloupec rok
df['ROK'] = df['DATE'].dt.year



In [15]:

output = widgets.Output()

def rocni_data(change):
    df1 = df.copy()
    df1 = df1.loc[df1['ROK'] == change['new']]

    
    with output:
        output.clear_output(wait=True)
        df1 = df1.groupby(by='NAME')['PRCP'].sum()
        print(df1)
        

vyber_rok = widgets.IntSlider(
    value = df['ROK'].max(),
    min = df['ROK'].min(),
    max = df['ROK'].max(),
    step = 1
)

vyber_rok.observe(rocni_data,names='value')


display(vyber_rok,output)

IntSlider(value=2020, max=2020, min=2000)

Output()